In [1]:
from data.dataset import Dataset

In [2]:
dataset = Dataset(data_path='./data/data_files/', n_samples=384, stress_number=2, load_number=2, augment=0, keep_prep=1)

Device:  cpu
<class 'int'>
384
<class 'torch.device'>
cpu
Preprocessing of file 1 out of 3
Value of n:  128 Value of i:  0
File 1 loaded
Input of file 1 saved. It corresponds to the young modulus.
Output of file 1 saved. It corresponds to the stress number 1 and the load number 1
Preprocessing of file 2 out of 3
Value of n:  128 Value of i:  1
File 2 loaded
Input of file 2 saved. It corresponds to the young modulus.
Output of file 2 saved. It corresponds to the stress number 1 and the load number 1
Preprocessing of file 3 out of 3
Value of n:  128 Value of i:  2
File 3 loaded
Input of file 3 saved. It corresponds to the young modulus.
Output of file 3 saved. It corresponds to the stress number 1 and the load number 1
